In [13]:
import sys
sys.path.append('..')
from exchanger.parts import *
from exchanger.stream import *
from exchanger.exchanger import*

Über das Konfigurationsfile `pyfluids.json` kann das Einheitensystem (für die Fluiddaten) festgelegt werden. Standartmäßg werden alle Daten in SI-Einheiten definiert und verarbeitet.

# Bsp Rohrbündelwärmeübertrager
### Angabe
Ein Rohrbundelwärmeubertrager mit einem äußeren und drei inneren Durchgängen (davon zwei im Gegenstrom) soll nachgerechnet werden. 
Folgende Daten zu den Fluidströmen sind bekannt:

Außenstrom:
+ Fluid: Luft
+ Massenstrom: $0,4$ kg/s
+ Eintrittstemperatur: $20$ °C
+ Druck: $1$ atm

Innenstrom:
+ Fluid: Wasser
+ Massenstrom: $0,15$ kg/s
+ Eintrittstemperatur: $105$ °C
+ Druck: $5$ bar

### Strömungsdaten
Zunächst werden die Ströme definiert. Dabei wird der Austrittsstrom zunächst bei derselben Temperatur wie am Eintritt festgelegt. Dadurch ergeben sich auch die mittleren Fluidparameter basierend auf den Eintrittsdaten. Da in späteren Schritten die Austrittstemperaturen angepasst werden und somit mehrere Iterationen mit den mittleren Temperaturen durchgeführt werden, sind anfängliche Abweichungen der Materialparameter nicht problematisch.

## Implementierung

Ein Fluidstrom der Klasse `Flow` wird über ein Fluid und einen Durchsatz definiert. Der Durchsatz kann entweder über den Massenstrom `mass_flow` oder Volumenstrom `volume_flow` festgelegt werden. Die Fluidinstanz wird durch die Klasse `Fluid` initialisiert. Dazu sind einerseits der Fluidname - dieser muss in der Fluidliste des Moduls `pyfluids` hinterlegt sein - sowie der Zustand erforderlich. Standardmäßig wird der Zustand mit der Methode `ntp_state` auf NTP (Normal Temperature and Pressure) festgelegt. Über die Eigenschaften `temperature` und `pressure` kann der Zustand angepasst werden.

Die Implementierung eines Fluidstroms erfolgt durch einen Volumenstrom - dabei wird angenommen, dass der Strömungsquerschnitt konstant ist - sowie einem Fluid am Ein- und Auslass. Die Initialisierung über den Konstruktor legt das Eintrittsfluid fest. Weiterhin wird der Volumenstrom festgelegt oder, sofern der Massenstrom initialisiert wird, basierend auf den Daten des Eintrittsfluids berechnet. Das Austrittsfluid wird zunächst als Klon des Eintrittsfluids definiert. Die Fluidstrom-Kennzahlen wie Wärmekapazitätstrom, Massenstrom und Änderung des Wärmestroms werden mit den Daten des Fluidobjekts `mean_fluid` berechnet und über gemittelte Zustandsdaten definiert.

Standardmäßig sind daher die Fluidparameter am Ein- und Auslass sowie die gemittelten Parameter gleich, und die Änderung des Wärmestroms (`heat_flow`) beträgt 0. Wenn die Austrittstemperatur (`out_temperature`) geändert oder ein Druckverlust (`pressure_loss`) definiert wird, werden auch die entsprechenden Parameter angepasst. Die Variable `phase_change` wird verwendet, um zu überprüfen, ob sich die Phase zwischen Ein- und Auslass ändert. Ist dies der Fall, sollte die Änderung des Wärmestroms in der aktuellen Implementierung nicht mehr für weitere Berechnungen verwendet werden.


In [14]:
# defining outside flow
flow_outside = Flow(Fluid("Air",temperature = 20+273.15),mass_flow=0.4)
flow_outside


Flow: id = 2521142047760
	mass flow = 0.40000 kg/s
	heat capacity flow: W_dot = 402.45761 W/K
	heat flow: Q_dot = 0.00000 kW

Input Fluid:
	Fluid: title = Air, id = 2521142192784
	p = 101325 Pa
	t = 20.0 °C
Output Fluid:
	Fluid: title = Air, id = 2521142046992
	p = 101325 Pa
	t = 20.0 °C

In [15]:
# defining inside flow
flow_inside = Flow(Fluid("Water",pressure=5e5,temperature=105+273.15),mass_flow=0.15)
flow_inside

Flow: id = 2521142047376
	mass flow = 0.15000 kg/s
	heat capacity flow: W_dot = 633.12137 W/K
	heat flow: Q_dot = 0.00000 kW

Input Fluid:
	Fluid: title = Water, id = 2521138527696
	p = 500000.0 Pa
	t = 105.0 °C
Output Fluid:
	Fluid: title = Water, id = 2521142047632
	p = 500000.0 Pa
	t = 105.0 °C

### Geometriedaten
Vom Aufbau des Wärmeübertragers sind folgende Daten bekannt.

Der Innenstrom ist auf $n = 20$ Rohrleitungen (Innendurchmesser $D_i = 8 mm$, Wandstärke $s = 2 mm$) aufgeteilt, welche in zwei Schichten versetzt angeordnet sind. Die Rohre haben außen kleine Strukturen, um bei der Umströmung eine turbulente Grenzschicht zu gewährleisten. Die folgenden weiteren Daten zur Geometrie sind bekannt (vgl. dazu die Skizzen in Abbildung):

+ $B = 0,35 m$
+ $H = 0,18 m$
+ $L_M = 1,0 m$
+ $L_E = 41 mm$
+ $d = 48.mm$

<img src="./images/simpleHeatexchanger.PNG" alt="heat_exchanger_geometrie" width="450" height="300">

Dadurch ist der verfügbare Bauraum zur Gänze ausgenutzt.

Der Wärmedurchgangskoezienten wird mit $k=35$ abgeschätzt.

## Implementierung

Für die Berechnung eines Wärmeübertrags ist in späterer Folge die Wärmeübertragungsfähigkeit relevant. Diese muss über die Klasse bzw. Subklasse `Part` übergeben werden. Die Instanz `Part` kann entweder direkt über die Parameter Wärmeübertragungsfähigkeit `heat_transferability` oder über die wärmeübertragende Fläche `heat_transfer_area` und den Wärmedurchgangskoeffizienten `heat_transfer_coefficient` definiert werden. Alternativ dazu können die Subklassen zur Berechnung, insbesondere der wärmeübertragenden Flächen - in erweiterten Implementierungen auch des Wärmedurchgangskoeffizienten - genutzt werden.

In diesem Beispiel wird die Länge eines Rohrs manuell kalkuliert. Damit kann zunächst ein einzelnes glattes Rohr `StraightPipe` über die Eigenschaften `diameter_in`, `diameter_out` und `length` definiert werden. Ein Rohrbündel wird über `PipeLayout` initialisiert und erfordert ein Objekt der Klasse `Pipe` sowie die Anzahl der Rohre. Damit kann die wärmeübertragende Fläche aus der eines einzelnen Rohrs und der Anzahl berechnet werden. Der Wärmedurchgangskoeffizient wird über `heat_transfer_coefficient` gesetzt. Damit ist die Wärmeübertragungsfähigkeit des Rohrbündels vollständig definiert.


In [16]:
l_m,l_e,d = 1,41e-3,48e-3
length = 3*l_m+2*l_e +d*pi

pipe = StraightPipe(diameter_in=11.9e-3,diameter_out=12e-3,length=length)
pipe_layout = PipeLayout(pipe,20)
pipe_layout.heat_transfer_coefficient = 35
pipe_layout

part:
	 id = 2521142046928
	 typ: PipeLayout

geometric properties:
	pipe type: StraightPipe
	length = 3.23280 m
	diameter: in = 0.01190 m, out = 0.01200 m
	number of pipes: 20
	pipe pattern: square

hydraulic properties:
	flow area = 0.002224 m^2

thermic properties:
	heat transferability: 84.956 W/K
	heat transfer area: 2.4273 m^2
	heat transfer coefficient: 35.00 W/(m^2 K)

### Wärmeübertager
Mit obigen Paramtern ergibt sich nach anpassen der Fluidparameter für einen Rohrbündelwärmeübertrager mit einem äußern und drei inneren durchgängen (zwei im Gegenstrom):

## Implementierung

Wenn die Austrittstemperatur eines Wärmeübertragers benötigt wird, für den bereits implementierte Formeln zur Verfügung stehen, kann dies über die Klasse `HeatExchanger` und ihre Subklassen erfolgen. Dazu sind die beiden Fluidströme der Klasse `Flow` sowie ein Bauteil der Klasse `Part` erforderlich.

Der Wärmetauscher wird also über die entsprechende Subklasse definiert, die dem gewünschten Typ entspricht. Zum Beispiel für einen reinen Gegenstromwärmetauscher `CounterCurrentFlow` oder für einen Rohrbündelwärmeübertrager mit einem äußeren und drei inneren Durchgängen (zwei im Gegenstrom) `OneOuterThreeInnerTwoCounterFlow`.

Mit dem Wärmekapazitätsstrom der Fluide `heat_capacity_flow`, der Wärmeübertragungsfähigkeit `heat_transferability` und den für den jeweiligen Typ entsprechenden Formeln werden dimensionslose Parameter wie die Anzahl der Transfer Units (NTU) `ntu`, dimensionslose Übertragungsfähigkeiten `r` und dimensionslose Temperaturänderungen `p` berechnet.

Die Austrittstemperatur wird über die dimensionslose Temperaturfähigkeit und die Temperatur des Eintrittsfluids `flow_1.in_fluid.temperature` bzw. `flow_2.in_fluid.temperature` des entsprechenden Fluidstroms mithilfe der Methode `_calc_output` berechnet. Gleichzeitig wird die Austrittstemperatur des Fluidstroms in fünf Iterationen angepasst und damit auch die anderen Strom- und Fluidparameter. Dadurch wird auch der übertragene Wärmestrom (sofern es zu keiner Phasenänderung kommt) entsprechend kalkuliert.

Standardmäßig ist das Attribut `auto_adjust` der Klasse `HeatExchanger` auf `True` gesetzt. Dadurch wird beim Ausführen der Methoden `str()` bzw. `print()` die Methode `_calc_output` automatisch aufgerufen.


In [17]:
ex = OneOuterThreeInnerTwoCounterFlow(flow_outside,flow_inside,pipe_layout)
ex.auto_adjust = False
ex


heat exchanger:
	id = 2521142088528
	type: OneOuterThreeInnerTwoCounterFlow
Flows:
Flow 1:
Flow: id = 2521142047760
	mass flow = 0.40000 kg/s
	heat capacity flow: W_dot = 402.45761 W/K
	heat flow: Q_dot = 0.00000 kW

Input Fluid:
	Fluid: title = Air, id = 2521142192784
	p = 101325 Pa
	t = 20.0 °C
Output Fluid:
	Fluid: title = Air, id = 2521142046992
	p = 101325 Pa
	t = 20.0 °C

Flow 2:
Flow: id = 2521142047376
	mass flow = 0.15000 kg/s
	heat capacity flow: W_dot = 633.12137 W/K
	heat flow: Q_dot = 0.00000 kW

Input Fluid:
	Fluid: title = Water, id = 2521138527696
	p = 500000.0 Pa
	t = 105.0 °C
Output Fluid:
	Fluid: title = Water, id = 2521142047632
	p = 500000.0 Pa
	t = 105.0 °C

Parameters:

dimensionless parameters:
number of transfer units:
	NTU_1 = 0.211
	NTU_2 = 0.134
heat capacity flow ratios:
	R_1 = 0.636
	R_2 = 1.573
dimensionless temperature change:
	P_1 = 0.179
	P_2 = 0.114

### Alternativer Wärmeübertrager Typ
Ein Wärmeübertrager mit reinem Gegenstrom und gleicher Wärmeübertragungsfähigkeit würde im gegeben Fall zu einer nahezu gleichen Austrittstemperatur führen.

Beim erhöhen der Übertragungsfähigkeit würde der Gegenstrom Wärmeübertrager einen besseren Wirkungsgrad ergeben. 

In [18]:
ex = CounterCurrentFlow(flow_outside,flow_inside,pipe_layout)
ex


heat exchanger:
	id = 2521142037200
	type: CounterCurrentFlow
Flows:
Flow 1:
Flow: id = 2521142047760
	mass flow = 0.38956 kg/s
	heat capacity flow: W_dot = 392.06020 W/K
	heat flow: Q_dot = -6.14130 kW

Input Fluid:
	Fluid: title = Air, id = 2521142192784
	p = 101325 Pa
	t = 20.0 °C
Output Fluid:
	Fluid: title = Air, id = 2521142046992
	p = 101325 Pa
	t = 35.66405412704643 °C

Flow 2:
Flow: id = 2521142047376
	mass flow = 0.15056 kg/s
	heat capacity flow: W_dot = 634.58189 W/K
	heat flow: Q_dot = 6.14138 kW

Input Fluid:
	Fluid: title = Water, id = 2521138527696
	p = 500000.0 Pa
	t = 105.0 °C
Output Fluid:
	Fluid: title = Water, id = 2521142047632
	p = 500000.0 Pa
	t = 95.32236458240772 °C

Parameters:

dimensionless parameters:
number of transfer units:
	NTU_1 = 0.217
	NTU_2 = 0.134
heat capacity flow ratios:
	R_1 = 0.618
	R_2 = 1.619
dimensionless temperature change:
	P_1 = 0.184
	P_2 = 0.114